In [13]:
import os
import cv2
import csv
import faiss
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from deepface import DeepFace
from deepface.modules import verification
from deepface.models.FacialRecognition import FacialRecognition

In [14]:
model_path = r'face_detection_yunet_2023mar.onnx'
face_detector = cv2.FaceDetectorYN_create(model=model_path, config='', nms_threshold=0.7,
                                        score_threshold=0.7, input_size=(0, 0), top_k=5000)
def detect_faces(image):
    h, w, _ = image.shape
    
    face_detector.setInputSize((w, h))
    n, faces = face_detector.detect(image)
    
    faces_details = {}
    if n > 0 and faces is not None:
        for f in faces:
            x, y, w, h = map(int, f[:4])
            faces_details[(x, y, w, h)] = image[y:y+h+1, x:x+w+1]
    return faces_details

In [15]:
dataset = 'Dataset'
persons = os.listdir(dataset)

target_size = (112, 112)
known_encodings = []
known_names = []

model: FacialRecognition = DeepFace.build_model(model_name='ArcFace')
threshold = verification.find_threshold('ArcFace', 'euclidean')

for person in os.listdir(dataset):
  person_folder = os.path.join(dataset, person)
  for person_img in os.listdir(person_folder):
    img_path = os.path.join(person_folder, person_img)
    image = cv2.imread(img_path)
    faces = detect_faces(image)
    if not faces:
      continue
    face_image = list(faces.values())[0]
    face_image = cv2.resize(face_image, target_size)
    face_image = np.expand_dims(face_image, axis = 0).astype('float32') / 255
    img_encoding = model.forward(face_image)
    known_encodings.append(img_encoding)
    known_names.append(person)

In [16]:
def take_attendance(name):
    os.makedirs("Attendance", exist_ok=True)
    file_path = f"Attendance/attendance_{datetime.now().date()}.csv"
    file_exists = os.path.exists(file_path)
    
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Name', 'Time'])
    
    name_exists = False
    with open(file_path, mode='r', newline='') as file:
        rows = csv.reader(file)
        for row in rows:
            if (len(row) > 0) and (name == row[0]):
                name_exists = True
                break
    
    with open(file_path, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    if not name_exists:
                        writer.writerow([name, datetime.now().strftime("%H:%M:%S")])

In [17]:
cap = cv2.VideoCapture(0)
dim = len(known_encodings[0])
index = faiss.IndexHNSWFlat(dim, 32)
index.add(np.array(known_encodings))
index.hnsw.efSearch = 16

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    
    faces = detect_faces(frame)
    if faces:
        for face in faces:
            face_img = faces[face]
            
            if 0 in face_img.shape[:2]:
                continue
            
            face_img = cv2.resize(face_img, target_size)
            face_img = np.expand_dims(face_img, axis = 0).astype('float32') / 255
            test_encoding = model.forward(face_img)

            d, i = index.search(np.expand_dims(test_encoding, axis = 0), 1)
            min_distance = np.sqrt(d[0, 0])
            min_distance_index = i[0, 0]

            if min_distance < threshold:               
                face_name = known_names[min_distance_index]
            else:
                face_name = "Unknown"
            
            if (face_name in persons):
                take_attendance(face_name)
            
            x, y, w, h = face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y - 20), (x+w, y), (0, 255, 0), -1)
            cv2.putText(frame, face_name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    
    cv2.imshow('Attendance System', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()